In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

cctv_seoul = pd.read_csv(r'./data/서울시 안심이 CCTV 연계 현황.csv', encoding='cp949')
pop_seoul  = pd.read_csv('.\data\등록인구(연령별_동별)_20250908141542.csv', encoding='utf-8')
crime_seoul= pd.read_csv(r'./data/5대+범죄+발생현황_20250908130333.csv', encoding='utf-8',
                         skiprows=2)


<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\paint\AppData\Local\Temp\ipykernel_10204\2109051646.py:7: SyntaxWarning: invalid escape sequence '\d'
  pop_seoul  = pd.read_csv('.\data\등록인구(연령별_동별)_20250908141542.csv', encoding='utf-8')


파일 위치 앞의 . 개수가 몇개의 상위 경로로 갈 지를 뜻함 ex) r'../데이터시각화/data~'

In [95]:
cctv_seoul.head()

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,중랑구,중화2동 322-16(고정2)_C-JH02-0003-B,37.5963,127.0739,1,2022-12-01
1,중랑구,신내2동 651(고정4)_929,37.6150,127.0889,1,2022-12-01
2,중랑구,중화2동 323-14 (초롱공원)_C-JH02-0023,37.5957,127.0732,1,2022-12-01
3,중랑구,신내2동 602-1 (시조목 관리 보조)_C-SN02-0001-A,37.6066,127.0919,1,2022-12-01
4,중랑구,중화2동 323-14(고정1)_C-JH02-0023-A,37.5957,127.0732,1,2022-12-01


안심 주소 언더바 뒤에 cctv의 id가 들어있음

In [96]:
cctv_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83734 entries, 0 to 83733
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   자치구      83734 non-null  object 
 1   안심 주소    83734 non-null  object 
 2   위도       83734 non-null  float64
 3   경도       83734 non-null  float64
 4   CCTV 수량  83734 non-null  int64  
 5   수정 일시    83734 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 3.8+ MB


In [97]:
data=pd.DataFrame()
data['구별']=cctv_seoul['자치구'].unique()
cctv_count=cctv_seoul.groupby('자치구').sum(['CCTV 수량'])
cctv_count.reset_index(inplace=True)
cctv_count.drop(['위도', '경도'], axis=1, inplace=True)
cctv_count=cctv_count.rename(columns={'자치구':'구별'})
data=pd.merge(data,cctv_count,on='구별')
data.head()

,구별,CCTV 수량
0,중랑구,3471
1,중구,2336
2,종로구,1930
3,은평구,4116
4,용산구,1078


In [98]:
cctv_gu_count=cctv_seoul.groupby('자치구').sum()['CCTV 수량'] #시리즈로 저장됨
cctv_gu=pd.DataFrame(cctv_gu_count) #데이터프레임으로 변환
cctv_gu=cctv_gu.reset_index() #인덱스를 컬럼으로 변환
cctv_gu=cctv_gu.rename(columns={'자치구':'구별'}) #컬럼명 변경

인덱스한테도 이름이 있다. 고로 멀티 인덱스 가능     
rename 할 때 inplace=True 옵션은 사용하지 말고 재정의 방법을 더 추천함      
method chaining을 사용하기 어려워지기 때문

In [99]:
pop_seoul.head(10)

,동별(1),항목,2025 2/4
0,동별(1),항목,합계
1,합계,계,9595509
2,합계,한국인,9325616
3,합계,등록외국인,269893
4,종로구,계,149332
5,종로구,한국인,138015
6,종로구,등록외국인,11317
7,중구,계,129616
8,중구,한국인,119097
9,중구,등록외국인,10519


iloc: 절대위치(인덱스 번호가 바껴도 위치에 따라 지정 가능)    
loc: 인덱스  

In [100]:
pop_seoul_gu=pop_seoul[pop_seoul['항목']=='계']
pop_seoul_gu.drop('항목',inplace=True, axis=1)
pop_seoul_gu=pop_seoul_gu.rename(columns={'동별(1)':'구별'})
pop_seoul_gu.reset_index(drop=True, inplace=True)
pop_seoul_gu= pop_seoul_gu.drop(index=0).reset_index(drop=True)
pop_seoul_gu=pop_seoul_gu.rename(columns={'2025 2/4':'인구수'})

C:\Users\paint\AppData\Local\Temp\ipykernel_10204\164031717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_seoul_gu.drop('항목',inplace=True, axis=1)


In [101]:
pop_seoul_gu.head()

,구별,인구수
0,종로구,149332
1,중구,129616
2,용산구,216214
3,성동구,280683
4,광진구,350164


In [102]:
data=pd.merge(data,pop_seoul_gu,on='구별')
data.head()

,구별,CCTV 수량,인구수
0,중랑구,3471,383958
1,중구,2336,129616
2,종로구,1930,149332
3,은평구,4116,463165
4,용산구,1078,216214


* 기준시점 : 연간은 매년 12.31일기준, 분기는 매분기 말기준      
* 분기(2025 2/4) 
* 세대수, 세대당인구 : 외국인 제외
* 65세이상 고령자 수 : 외국인 포함

In [103]:
crime_seoul.head()

,자치구별(1),자치구별(2),2023,2023.1,2023.2,2023.3,2023.4,2023.5,2023.6,2023.7,2023.8,2023.9,2023.10,2023.11
0,자치구별(1),자치구별(2),합계,합계,합계,합계,합계,합계,합계,합계,합계,합계,합계,합계
1,자치구별(1),자치구별(2),소계,소계,살인,살인,강도,강도,강간·강제추행,강간·강제추행,절도,절도,폭력,폭력
2,자치구별(1),자치구별(2),발생,검거,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거
3,합계,소계,86213,64823,150,144,129,117,5764,5395,37038,21600,43132,37567
4,합계,종로구,2981,3485,5,6,5,7,239,1138,1169,953,1563,1381


In [105]:
crime_seoul.columns = crime_seoul.columns.str.replace(r"^2023.*", "2023", regex=True)
crime_seoul.head()

,자치구별(1),자치구별(2),2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023
0,자치구별(1),자치구별(2),합계,합계,합계,합계,합계,합계,합계,합계,합계,합계,합계,합계
1,자치구별(1),자치구별(2),소계,소계,살인,살인,강도,강도,강간·강제추행,강간·강제추행,절도,절도,폭력,폭력
2,자치구별(1),자치구별(2),발생,검거,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거
3,합계,소계,86213,64823,150,144,129,117,5764,5395,37038,21600,43132,37567
4,합계,종로구,2981,3485,5,6,5,7,239,1138,1169,953,1563,1381


In [ ]:
crime_seoul.columns = crime_seoul.iloc[1]

,자치구별(1),자치구별(2),2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023
0,자치구별(1),자치구별(2),발생,검거,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거
1,합계,소계,86213,64823,150,144,129,117,5764,5395,37038,21600,43132,37567
2,합계,종로구,2981,3485,5,6,5,7,239,1138,1169,953,1563,1381
3,합계,중구,3348,2465,3,3,6,5,212,144,1520,864,1607,1449
4,합계,용산구,3021,2268,3,4,3,3,245,219,978,561,1792,1481


In [ ]:
# 첫 번째 행을 컬럼명으로 지정
crime_seoul.columns = crime_seoul.iloc[0]

# 첫 번째 행은 컬럼명으로 올렸으니 데이터에서는 제외
crime_seoul = crime_seoul.drop(index=0).reset_index(drop=True)